# Vessel Density based on Copernicus Sentinel-1
---
_Author: Alessandro Cimbelli, Oct 2022_

In this updated proposal, the marine area of interest is divided and analysed into areas of 100 km x 100 km
#### Methodology 
1. Definition of the boxes ID to be analysed.
2. Extraction of the footprints of the Sentinel-1 (VV band) images acquired in the time period (one month).
3. Extraction of vessel centroids in each image
4. Map of vessel density for each 1-km grid tile of the selected area
5. Production of geojson and csv files


In [1]:
year = 2020
months = [1,2,3,4,5,6,7,8,9,10,11,12]

In [2]:
#from edc import check_compatibility
#check_compatibility("user-0.24.5")

In [3]:
#from edc import setup_environment_variables
#setup_environment_variables()

In [4]:
from xcube_sh.config import CubeConfig
from xcube_sh.cube import open_cube
from xcube_sh.sentinelhub import SentinelHub
from xcube.core.geom import (
    clip_dataset_by_geometry, 
    mask_dataset_by_geometry,
    convert_geometry,
    rasterize_features
)
import xarray as xr
import numpy as np
import shapely.geometry
import IPython.display
import calendar

import datetime
import os, sys
%matplotlib inline
import matplotlib.pyplot as plt
import rasterio, rioxarray
import geopandas as gpd
import pandas as pd
import warnings
import scipy
import pyproj
from shapely.ops import transform, cascaded_union, unary_union
from shapely.geometry import shape, Point, Polygon, LineString, MultiPoint, MultiPolygon, mapping 
from rasterio.features import sieve, shapes
from rasterio.plot import show, plotting_extent
from rasterio.enums import Resampling
import json
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import requests
import csv
import shutil

In [5]:
# Your client credentials
client_id = os.environ['SH_CLIENT_ID']
client_secret = os.environ['SH_CLIENT_SECRET']

grid_name = "grid100k_geo.geojson"
grid100k = gpd.read_file(grid_name)

res1 = 0.0054 # 0.00054 # lower resolution for satellite footprints
res2 = 0.00018 # Higher resolution for vessel shape and number
#dres = (res1-res2)/2
###############################
# csv parameters
eo_sensor = 'Sentinel-1'
indicator_code = 'E1b'
y_axis = 'Vessel density'
indicator_name = 'Vessel density'
data_provider = "cimbelli"
description = 'Number of vessels in the AOI'
method = 'Object detection of vessels from Sentinel-1 images'
update_freq = 'Monthly'

In [6]:
# This function extracts the centroids of all the vessels in an image
def vesselShape(imgtif):

    img1tmp = imgtif.astype(rasterio.uint8)
    img2tmp = rasterio.features.sieve(img1tmp.to_numpy(), 15, connectivity=8)
    
    with rasterio.Env():
        image = img2tmp #img_vessels.read(1) # first band
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, mask=None, transform=imgtif.rio.transform())))
        geoms = list(results)
    npoly = len(geoms)
    npoly_1 = 0
    
# for each vessel get shape information
    
    ppc = MultiPoint([])
    if npoly > 0:
        for p in range(npoly):
            val = int(geoms[p]['properties']['raster_val'])

            if val==1:
                npoly_1 += 1
                pp = shape(geoms[p]['geometry'])
                pp_centroids = pp.centroid #.coords     
                ppc = ppc.union(pp_centroids)   
        return ppc, npoly_1
    else:
        return None, 0
   

In [7]:
def sentinelInfo(data, box, n):

    # Create a session
    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client)

    # Get token for the session
    token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token', client_secret=client_secret)

    # All requests using this session will have an access token automatically added
    resp = oauth.get("https://services.sentinel-hub.com/oauth/tokeninfo") 
    token1 = token['access_token']
    
    data = '{"collections":["sentinel-1-grd"], "datetime":"' + data + '","bbox": [' + box + '] ,"limit": ' + str(n) + '}'  # 2020-01-01T00:00:00Z/2020-01-31T23:59:59Z - 10,54.27,11,54.6
    headers = {"Content-Type": "application/json" , "Authorization": "Bearer " + token1 }
    endpoint = "https://services.sentinel-hub.com/api/v1/catalog/search"
    
    ggg = requests.post(endpoint, data = data, headers=headers)
    
    res = json.loads(ggg.content.decode('UTF-8'))
    id = res['features'][0]['id']
    sat = id[2:3]
    
    ddatetime = res['features'][0]['properties']['datetime'][:-1]
    
    return sat, ddatetime 
 

### Definition of the boxes ID to be analysed. Please edit them in the list below the map.

_The mask of all European water areas has been produced from the coastline shapefile available at the page:_

https://www.eea.europa.eu/data-and-maps/data/eea-coastline-for-analysis-1/gis-data/europe-coastline-shapefile


In [8]:
%%html
<style>
.leaflet-div-icon { background-color: transparent; 
                   border-color: transparent; 
                   text-align: center; 
                   align-items: center
                  }
.leaflet-container::after{ text-align: center; }
.leaflet-popup-content img { margin: 0 auto; display:block; }
</style>

In [9]:
with open('grid100k_geo.geojson', 'r') as f:
    data = json.load(f)

from ipyleaflet import Map, basemaps, basemap_to_tiles, GeoJSON, Marker, DivIcon, WidgetControl
m = Map(
    basemap = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center = (49.0, 9.0),
    zoom = 5
    )
geo_json = GeoJSON(
    data = data,
    style = {'opacity':1, 'fillOpacity':0.1, 'weight':1, 'fillColor':'green'},
    hover_style = {'color':'blue', 'fillOpacity':0.5, 'weight':1.2}
    )
for i in range(0,len(grid100k)):
    extx = grid100k.iloc[i]["geometry"].bounds[2]-grid100k.iloc[i]["geometry"].bounds[0]
    lo = grid100k.iloc[i]["geometry"].bounds[0] + extx*0.25  #grid100k.iloc[i]["geometry"].centroid.x
    la = grid100k.iloc[i]["geometry"].centroid.y
    marker = Marker(
        location = [la, lo],    
        icon = DivIcon(html=f"""<div style="font-family: calibri; color: blue; font-size: 90%; font-weight: bold">{grid100k.iloc[i]['Aoi_Id']}</div>""",),
        draggable=False
    )
    m.add_layer(marker)
m.add_layer(geo_json)
m.layout.height = '800px'
m

Map(center=[49.0, 9.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

In [10]:
# TILE LIST

# 'BE1','DE1','DE2','DK2','EE1','GR7',
# 'ES1','ES18','ES3','ES4','ES5','FR2',
# 'FR3','FR4','IE2','IT3','IT16','IT17',
# 'IT18','IT19','LT1','LV1','NL3','NL4',
# 'NO3','NO4','PL1','PT2','RO1','RU1',
# 'TR1','TR2'

bbox_list = ('NL3',)



In [11]:
######################################################
# Density of vessels for each 1km cell
######################################################

warnings.filterwarnings('ignore')
# generate csv

gen_date = datetime.datetime.today().strftime('%Y%m%dT%H%M%S')
csv_file = indicator_code + '_' + data_provider + '_' +  gen_date + '.csv'
header = ['AOI','Country','Region','City','Site Name','Description','Method','EO Sensor','Input Data', 
          'Indicator code','Time','Measurement Value','Reference Description','Reference time','Reference value','Rule', 
          'Indicator Value','Sub-AOI','Y axis','Indicator Name','Color code','Data Provider','AOI_ID','Update Frequency']

if os.path.exists('out/' + csv_file):
    os.remove('out/' + csv_file) 
with open('out/' + csv_file, 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)

# start processing

    for b in range(len(bbox_list)):
      
        for month in months:
            
            
            d1 = datetime.datetime(year, month, 1,12,0,0)
            d2 = datetime.datetime(year, month, calendar.monthrange(year, month)[1],12,0,0)
            time_interval= d1.strftime("%Y-%m-%d"),d2.strftime("%Y-%m-%d")
            month = str(month) if (month)>9 else "0"+ str(month)
            
            arr_s1 = []
            poly = grid100k[grid100k.Aoi_Id == bbox_list[b]]

            # --------------------------
            # csv file
            id100 = poly.id_100k.values[0]
            aoi_center = str(round(poly.geometry.centroid.values[0].y,6)) + ", " + str(round(poly.geometry.centroid.values[0].x,6))
            co = poly.Country.values[0]
            port = poly.Port.values[0]
            city = poly.City.values[0]
            bbox = str(round(poly.geometry.bounds.values[0][0],3)) + ',' + str(round(poly.geometry.bounds.values[0][1],3)) + ',' + str(round(poly.geometry.bounds.values[0][2],3)) + ',' + str(round(poly.geometry.bounds.values[0][3],3))  

            # ---------------------------------------------
            tmp_masks = []
            tmp_masks.clear() 
            # ---------------------------------------------
            config_s1 = CubeConfig(dataset_name = 'S1GRD',
                                     band_names = ['VV'],
                                     tile_size = [512, 512],
                                     crs = "http://www.opengis.net/def/crs/EPSG/0/4326",
                                     spatial_res =  res2, 
                                     bbox = bbox,
                                     time_range = time_interval, 
                                  ) 
            s1 = open_cube(config_s1)
            n_img = len(s1.VV.time)
            print("aoi", bbox_list[b], " - ",n_img, 'images in the month of', month, '-',year)
            print("   day")   
            # ---------------------------------------------
            points = MultiPoint([])   
            
            for j in range(n_img):  

                pp1 = pd.DataFrame()
                img0 = s1.VV[j]

                # ---------------------------------------------
                # for footprints
                tmp_mask = img0.notnull() 
                tmp_mask = tmp_mask.astype(rasterio.uint8)
                tmp_mask.rio.write_crs("EPSG:4326", inplace=True)

                tmp_masks.append(tmp_mask)

                # ---------------------------------------------
                # define geojson filename
                daytime = str(s1.VV.time[j].values).replace("-","").replace(":","")
                daytime = daytime[:daytime.find(".")]
                daytime1 = str(s1.VV.time[j].values).replace("-","/").replace("T"," ")
                daytime1 = daytime1[:daytime1.find(".")]

                day = daytime[6:11]
                hour = int(day[-2:])
                hour1 = str(hour) if (hour)>9 else "0"+ str(hour)
                print("      ", day, end="..")


                # mask image with land and fixed semarks (windfarms, ...)
                img1 = img0 > 0.7
                img2 = img1.astype(rasterio.uint8)
                img2.rio.write_crs("EPSG:4326", inplace=True)

                land = gpd.read_file('sea_4326.geojson')
                clipped = img2.rio.clip(land.geometry, land.crs, drop=True, invert=False)

                # assess the number of vessels
                pp, nvess = vesselShape(clipped)  
                print(nvess, "vessels")

                # ---------------------------
                # geojson
                # save vessels' centroids from a single image to a geojson file        

                if not(pp.is_empty):
                    if type(pp) == shapely.geometry.point.Point:   # only 1 vessel
                        points1 = [Point(pp.x, pp.y)]
                        pp1 = gpd.GeoDataFrame(geometry=points1, crs=4326)
                        pp1.rename(columns ={'geometry':'0'}) 
                    elif len(pp)>0:
                        pp1 = gpd.GeoDataFrame(pp, geometry=0, crs=4326)   #'EPSG:4326')


                    # only if there are vessels
                    if len(pp1)>0:

                        # ===================== csv ========================== #

                        daytime2 = daytime1[:10]
                        daytime2 = daytime2.replace('/', '-') + "T" + hour1 + ":00:00Z/" + daytime2.replace('/', '-') + "T" + hour1 + ":59:59Z"

                        # it could happen 2 images in the same day....

                        ssat, ddate = sentinelInfo(daytime2, bbox, 1)
                        input_data = 'S1' + ssat +' - GRD'
                        measurement_value = len(pp1)
                        indicator_value = pp.wkt

                        #['AOI','Country','Region','City','Site Name','Description','Method','EO Sensor','Input Data', 
                        #'Indicator code','Time','Measurement Value','Reference Description','Reference time','Reference value','Rule', 
                        #'Indicator Value','Sub-AOI','Y axis','Indicator Name','Color code','Data Provider','AOI_ID','Update Frequency']

                        info = [aoi_center,co,'',city,port,description,method,eo_sensor,input_data,
                                indicator_code,ddate,measurement_value,'',ddate,'','','',
                                poly.geometry.values[0],'',indicator_name,'RED',data_provider,bbox_list[b],update_freq]
                        writer.writerow(info)

                        # ---------------------------

                        # geojson 

                        date_geojson = ddate.replace("-","/").replace("T"," ")
                        date_geojson1 = ddate.replace("-","").replace(":","")
                        pp1['TIMESTAMP UTC'] = date_geojson
                        pp1['CLASSIFICATION'] = "Vessel"

                        geojson_file = indicator_code + "_" + str(bbox_list[b]) + "_" + date_geojson1 + ".geojson"
                        #print(daytime, geojson_file)

                        if os.path.exists('out/' + geojson_file):
                            os.remove('out/' + geojson_file) 
                        #pp1.to_crs('EPSG:3035').to_file('out/' + geojson_file, driver='GeoJSON')
                        pp1.to_crs('EPSG:4326').to_file('out/' + geojson_file, driver='GeoJSON')

                        with open('out/' + geojson_file, 'r') as file:
                            filedata = file.read()
                            filedata = filedata.replace('/', '\/')
                            #remove info on crs
                            filedata = filedata.replace('"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },\n', '')

                        with open('out/' + geojson_file, 'w') as file:
                            file.write(filedata)
                        # ---------------------------

                        points = points.union(pp) 

                    else:
                        #nv = 0
                        #print("n", end="..")
                        continue

                else:
                    #nv = 0
                    #print("n", end="..")
                    continue

            # ---------------------------------------------
            # for footprints
            print("   footprints")
            mask = xr.concat(tmp_masks, dim='time')
            mask2 = mask.sum(dim='time')
            mask2 = mask2.astype(rasterio.uint8)
            mask2.rio.write_crs("EPSG:4326", inplace=True)

            #vectorize raster of footprints
            with rasterio.Env():     
                results = (
                {'properties': {'raster_val': v}, 'geometry': s}
                for i, (s, v) 
                in enumerate(
                    shapes(mask2.values, mask=None, connectivity=4, transform=mask2.rio.transform())))
                ftprints = list(results)

            #---------------------------------------------------------
            # create geojson from footprints

            tmp_list = []

            for p2 in range(len(ftprints)):
                val = int(ftprints[p2]['properties']['raster_val'])
                ppp2 = shape(ftprints[p2]['geometry']) 
                tmp_list.append({
                  'geometry' : Polygon(ppp2),
                  'val': val
                 })
            mpl = gpd.GeoDataFrame(tmp_list, crs='epsg:4326')
            mpl.to_file("mpl.geojson", driver='GeoJSON')    

            # ---------------------------------------------

            ppp = gpd.GeoDataFrame(points,geometry=0, crs='EPSG:4326')
            ppp.insert(1, 't', 0)
            ppp.rename_geometry('geometry').to_file("vvv.geojson", driver='GeoJSON')
            points3 = ppp.to_crs('EPSG:3035')

            # count vessels for each vector grid with 1-km pixel
            
            print("   vessel count")
            grid1k = gpd.read_file('grid1k/'+ bbox_list[b] + '.geojson')
            if os.path.exists('fin.geojson'):
                os.remove('fin.geojson')
            dfsjoin = gpd.sjoin(grid1k, points3,how="inner") #,predicate="within"
            dfpivot = pd.pivot_table(dfsjoin,index='id',columns='t',aggfunc={'t':len})
            dfpivot.columns = dfpivot.columns.droplevel()
            dfpolynew = grid1k.merge(dfpivot, how='left', on='id')
            df = dfpolynew.rename(columns ={dfpolynew.columns[-1]:'vessels'})
            df.columns = df.columns.astype(str)
            df.to_file("fin.geojson", driver='GeoJSON')
 
            # convert vector grid to raster and calc density
            print("   density")

            # create rasters of number of vessels and acquisitions
            raster1k = 'grid1k/'+ bbox_list[b] + '.tif'
            !gdal_calc.py --quiet --overwrite -A $raster1k --outfile=n_vessels.tif --calc="0"
            shutil.copy('n_vessels.tif', 'n_acquisitions.tif')
            !gdal_rasterize -q -add -a vessels fin.geojson n_vessels.tif
            !gdal_rasterize -q -add -a val mpl.geojson n_acquisitions.tif


            # raster of the vessel density
            dens_temp = "out/density_vessels_temp.tif"

            dens = "out/density_vessels_" + bbox_list[b] + "_" + str(year) + str(month) + '01_cog.tiff'
            !gdal_calc.py --quiet --overwrite -A n_vessels.tif -B n_acquisitions.tif --outfile=$dens_temp --calc="(1000*A)/B"
            !gdal_translate -q -of COG -co COMPRESS=DEFLATE -co BLOCKSIZE=1024 -co RESAMPLING=AVERAGE -co OVERVIEWS=IGNORE_EXISTING -ot Int16 -a_nodata -9999 $dens_temp $dens


            if os.path.exists(dens_temp):
                os.remove(dens_temp)
            if os.path.exists('fin.geojson'):
                os.remove('fin.geojson')
            if os.path.exists('vvv.geojson'):
                os.remove('vvv.geojson') 
            print("   ok")


print("ok")

SentinelHubError: 403 Client Error: Forbidden for url: https://services.sentinel-hub.com/api/v1/catalog/search

In [6]:
shutil.make_archive("output",'zip',"out")

'/home/jovyan/output.zip'

In [13]:
shutil.make_archive("grid1k",'zip',"grid1k")

'/home/jovyan/grid1k.zip'